In [1]:
import dash
from jupyter_dash import JupyterDash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input,Output
import plotly.express as px
import dash_table
import pandas as pd
import geocoder as g
import requests
import plotly.graph_objects as go
from IPython.display import display


In [2]:
hakuurl="https://raw.githubusercontent.com/varmais/maakunnat/master/kunnat.geojson"
haku=requests.get(url=hakuurl, timeout=60000)
geojson=haku.json()

In [3]:
df=pd.read_excel('Sukupuoliikä.xlsx')
df

,Alue,Ikä,Sukupuoli,Vuosi,Väestö 31.12.
0,Forssa,Yhteensä,Yhteensä,2000,18506
1,Forssa,Yhteensä,Yhteensä,2001,18311
2,Forssa,Yhteensä,Yhteensä,2002,18201
3,Forssa,Yhteensä,Yhteensä,2003,18104
4,Forssa,Yhteensä,Yhteensä,2004,18076
...,...,...,...,...,...
4795,Sysmä,65 -,Naiset,2015,818
4796,Sysmä,65 -,Naiset,2016,823
4797,Sysmä,65 -,Naiset,2017,818
4798,Sysmä,65 -,Naiset,2018,815


In [4]:
df2=pd.read_excel('2000-2019väestöyhteensä.xlsx')
df2 = df2.melt(id_vars=["Alue","Vuosi"], value_vars=["Miehet Väestö 31.12.", "Naiset Väestö 31.12.", "Yhteensä Väestö 31.12."], var_name="sukupuoli", value_name="henkilöt")
df2['Yhteensäerotus'] = df2['henkilöt'].diff()

df2

,Alue,Vuosi,sukupuoli,henkilöt,Yhteensäerotus
0,Forssa,2000,Miehet Väestö 31.12.,8913,NaN
1,Forssa,2001,Miehet Väestö 31.12.,8829,-84.0
2,Forssa,2002,Miehet Väestö 31.12.,8832,3.0
3,Forssa,2003,Miehet Väestö 31.12.,8750,-82.0
4,Forssa,2004,Miehet Väestö 31.12.,8737,-13.0
...,...,...,...,...,...
1195,Sysmä,2015,Yhteensä Väestö 31.12.,4040,-57.0
1196,Sysmä,2016,Yhteensä Väestö 31.12.,3953,-87.0
1197,Sysmä,2017,Yhteensä Väestö 31.12.,3859,-94.0
1198,Sysmä,2018,Yhteensä Väestö 31.12.,3753,-106.0


In [38]:
df3 = df[df.Sukupuoli != "Yhteensä"]
df3=df3[df3.Ikä != "Yhteensä"]
df3

,Alue,Ikä,Sukupuoli,Vuosi,Väestö 31.12.
80,Forssa,0 - 14,Miehet,2000,1495
81,Forssa,0 - 14,Miehet,2001,1493
82,Forssa,0 - 14,Miehet,2002,1448
83,Forssa,0 - 14,Miehet,2003,1407
84,Forssa,0 - 14,Miehet,2004,1399
...,...,...,...,...,...
4795,Sysmä,65 -,Naiset,2015,818
4796,Sysmä,65 -,Naiset,2016,823
4797,Sysmä,65 -,Naiset,2017,818
4798,Sysmä,65 -,Naiset,2018,815


In [5]:
Sukupuoli = df.Sukupuoli.unique()
Ikä = df.Ikä.unique()
Vuosi=df.Vuosi.unique()
Alue=df.Alue.unique()


In [43]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

colors = {
    "graphBackground": "#F5F5F5",
    "background": "#ffffff",
    "text": "#123"
}


app.layout=html.Div(children =[
    html.H3("Kalle Karlsson, XAMK"),
    html.H1(
        children='Verkkosivu visualisointeja varten',
        style={
            'textAlign': 'center','color': '#7FDBFF'}),
    html.Div([
    html.H1(
        children='teksti tähän',
        style={'width': '100%','height': '150%', 'display': 'inline-block'})]),

    html.Div([
        html.H4('Valitse ikäryhmä alemmasta valikosta ja voit myös suodattaa maakunnan oikealla olevasta legendasta tuplaklikkaamalla!'),
        
        dcc.Dropdown(id="Ikä", 
                     options=[{"label": x, "value": x} for x in Ikä], 
                     value=Ikä[0],
                     clearable=False),
        
        dcc.Graph(id="graph")],
        
        style={'width': '100%', 'display': 'inline-block'}),
        
    html.Div([    
        html.H1("Demo:Plotly")]),

    html.Div([
        dcc.Graph(id="graph2", figure={}),
       
        dcc.Slider(id='slider',
                   min=df2['Vuosi'][1],
                   max=df2['Vuosi'].max(),
                   value=df2['Vuosi'][1],
                   marks={str(year): str(year) for year in df2['Vuosi'].unique()},
                   step=None)],
        
        style={'width': '100%', 'display': 'inline-block'}),
  
    
    html.Div([
        html.H1("Teksti tähän"),
        dcc.Dropdown( id="dropdown",
                     options=[{"label": x, "value": x} for x in Alue],
                     value=Alue[0],
                     clearable=False),
        
        dcc.Graph(id="graph3")],
        
        style={'width': '100%', 'display': 'inline-block'})

])


@app.callback(Output('graph','figure'), 
              Input('Ikä', 'value'))

def display_selected_year(Ikä):
    mask = df["Ikä"] == Ikä
    fig = px.scatter(df[mask], x="Vuosi", y="Väestö 31.12.", color="Alue",symbol="Sukupuoli",log_x=True,
                     labels={"Sukupuoli": "Sukupuoli"},width=2000, height=1000, hover_name="Ikä")
    fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

    return fig

    
                    
@app.callback(Output('graph2','figure'), 
              Input('slider', 'value'))

def update_figure(selected_year):
    filtered_df = df2[df2.Vuosi == selected_year]

    fig=px.choropleth(filtered_df,width=2000, height=700, geojson=geojson, color="Yhteensäerotus", 
                    locations="Alue", featureidkey="properties.Kunta",color_continuous_scale="Viridis",
                    projection="mercator", facet_col="sukupuoli", title="Sukupuoli",
                    labels=dict(Yhteensäerotus="Vuosittainen muutos", Alue="Maakunta",range_color=[-500, 500]))
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.update_layout(hovermode="x unified")
    


    return fig

@app.callback(
    Output("graph3", "figure"), 
    [Input("dropdown", "value")])

def update_bar_chart(Alue):
    mask = df3["Alue"] == Alue
    fig = px.bar(df3[mask], x="Alue", y="Väestö 31.12.", barmode="group", facet_col="Sukupuoli",
                 color="Ikä", width=2000, height=1000,
                 animation_frame="Vuosi", animation_group="Alue",
             category_orders={"Ikä": ["Thur", "Fri", "Sat", "Sun"],
                              "Sukupuoli": ["Lunch", "Dinner"]})
    return fig

app.run_server(mode="jupyterLab", port=8051)